In [ ]:
# Optional: install CPU TensorFlow + deps (uncomment if needed)
# %pip install -q tensorflow-cpu==2.15.0 keras==2.15.0 tensorflow-io-gcs-filesystem==0.31.0 \
#                 pillow matplotlib seaborn scikit-learn opencv-python
import sys, platform, os, json
from pathlib import Path
print('Python:', sys.version)
print('Platform:', platform.platform())

In [ ]:
# Resolve project paths (robust to where the notebook is opened)
from pathlib import Path
def find_ml_service_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(10):
        if (p / 'ml_service').exists():
            return (p / 'ml_service')
        if p.name == 'ml_service':
            return p
        p = p.parent
    return start.resolve()
cwd = Path.cwd()
ml_root = find_ml_service_root(cwd)
print('ml_root =', ml_root)
data_root = ml_root / 'dataset' / 'Autism emotion recogition dataset' / 'Autism emotion recogition dataset'
train_dir = data_root / 'train'
test_dir = data_root / 'test'
models_dir = ml_root / 'models'
models_dir.mkdir(parents=True, exist_ok=True)
best_model_path = models_dir / 'best_model.keras'
label_map_path = models_dir / 'label_map.json'
print('train_dir =', train_dir)
print('test_dir  =', test_dir)
print('models_dir=', models_dir)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import itertools
IMG_SIZE = (224, 224)
BATCH = 32
VAL_SPLIT = 0.2
SEED = 42
CLASSES = ['Natural','joy','fear','anger','sadness','surprise']  # fixed order

In [ ]:
# Build datasets using directory structure
assert train_dir.exists(), f'Train directory not found: {train_dir}'
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=VAL_SPLIT,
    subset='training',
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    label_mode='int',
    class_names=CLASSES
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=VAL_SPLIT,
    subset='validation',
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH,
    label_mode='int',
    class_names=CLASSES
)
class_names = CLASSES
print('Class names:', class_names)
with open(label_map_path, 'w', encoding='utf-8') as f:
    json.dump({'classes': class_names}, f, indent=2)
# Prefetch
AUTOTUNE = tf.data.AUTOTUNE
def prep(ds):
    return ds.map(lambda x,y: (preprocess_input(tf.cast(x, tf.float32)), y), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
train_ds_p = prep(train_ds)
val_ds_p = prep(val_ds)
test_ds_p = None
if test_dir.exists():
    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        test_dir, image_size=IMG_SIZE, batch_size=BATCH, label_mode='int', class_names=CLASSES)
    test_ds_p = prep(test_ds)
    print('Test samples available')
else:
    print('No separate test directory; will evaluate on validation set.')

In [ ]:
# Build DenseNet-121 model
base = DenseNet121(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))
base.trainable = False  # fine-tune later if needed
inputs = layers.Input(shape=IMG_SIZE + (3,))
x = preprocess_input(tf.cast(inputs, tf.float32))
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(CLASSES), activation='softmax')(x)
model = models.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
# Train
ckpt = tf.keras.callbacks.ModelCheckpoint(str(best_model_path), monitor='val_accuracy', save_best_only=True, verbose=1)
early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
hist = model.fit(train_ds_p, validation_data=val_ds_p, epochs=15, callbacks=[ckpt, early])
# Ensure best is saved
model.save(best_model_path)
print('Saved best model to:', best_model_path)
# Write BEST_MODEL_PATH.txt for the Flask app
with open(ml_root / 'BEST_MODEL_PATH.txt', 'w', encoding='utf-8') as f:
    f.write(str(best_model_path))

In [ ]:
# Evaluate
eval_ds = test_ds_p or val_ds_p
loss, acc = model.evaluate(eval_ds)
print('Eval accuracy:', acc)
# Classification report
y_true, y_pred = [], []
for bx, by in eval_ds:
    p = model.predict(bx, verbose=0)
    y_true.extend(by.numpy().tolist())
    y_pred.extend(np.argmax(p, axis=1).tolist())
print(classification_report(y_true, y_pred, target_names=CLASSES))
cm = confusion_matrix(y_true, y_pred)
cm

In [ ]:
# Confusion matrix visualization
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES, ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('True')
plt.tight_layout()
plt.show()

In [ ]:
# Single-image inference helper
from PIL import Image
def predict_image(path: str):
    img = Image.open(path).convert('RGB').resize(IMG_SIZE)
    arr = np.array(img)[None, ...]
    arr = preprocess_input(arr.astype('float32'))
    probs = model.predict(arr, verbose=0)[0]
    top = int(np.argmax(probs))
    return {
        'emotion': CLASSES[top],
        'confidence': float(probs[top]),
        'allPredictions': {c: float(probs[i]) for i, c in enumerate(CLASSES)}
    }
# Example: replace with your image path
# predict_image(str(train_dir / 'joy' / os.listdir(train_dir / 'joy')[0]))